In [1]:
import argparse
import sys
import os
import shutil
import tempfile
import mlflow
from mlflow import pyfunc, tensorflow
import pandas as pd
import tensorflow as tf

In [3]:
batch_size = 100
train_steps = 1000

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
mlflow.set_experiment('experiment-tf2')

In [5]:
def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features = dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [6]:
with mlflow.start_run():
    tensorflow.autolog()
    #mlflow.set_tracking_uri("http://mlflow:5000")
    # Fetch the data
    (train_x, train_y), (test_x, test_y) = load_data()
    
    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in list(train_x.keys()):
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Two hidden layers of 10 nodes each.
    hidden_units = [10, 10]

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=hidden_units,
        # The model must choose between 3 classes.
        n_classes=3)

    # Train the Model.
    classifier.train(
        input_fn=lambda: train_input_fn(train_x, train_y,
                                        batch_size),
        steps=train_steps)

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda: eval_input_fn(test_x, test_y,
                                       batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # Generate predictions from the model
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda: eval_input_fn(predict_x,
                                       labels=None,
                                       batch_size=batch_size))

    old_predictions = []
    template = '\nPrediction is "{}" ({:.1f}%), expected "{}"'

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(SPECIES[class_id],
                              100 * probability, expec))

        old_predictions.append(SPECIES[class_id])

    # Creating output tf.Variables to specify the output of the saved model.
    feat_specifications = {
        'SepalLength': tf.Variable([], dtype=tf.float64, name="SepalLength"),
        'SepalWidth': tf.Variable([], dtype=tf.float64, name="SepalWidth"),
        'PetalLength': tf.Variable([], dtype=tf.float64, name="PetalLength"),
        'PetalWidth': tf.Variable([], dtype=tf.float64, name="PetalWidth")
    }

    receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feat_specifications)
    temp = tempfile.mkdtemp()
    try:
        # The model is automatically logged when export_saved_model() is called.
        saved_estimator_path = classifier.export_saved_model(temp, receiver_fn).decode("utf-8")
        print(saved_estimator_path)
        # Since the model was automatically logged as an artifact (more specifically
        # a MLflow Model), we don't need to use saved_estimator_path to load back the model.
        # MLflow takes care of it!
        pyfunc_model = pyfunc.load_model(mlflow.get_artifact_uri('model'))

        predict_data = [[5.1, 3.3, 1.7, 0.5], [5.9, 3.0, 4.2, 1.5], [6.9, 3.1, 5.4, 2.1]]
        df = pd.DataFrame(data=predict_data, columns=["SepalLength", "SepalWidth",
                                                      "PetalLength", "PetalWidth"])

        # Predicting on the loaded Python Function and a DataFrame containing the
        # original data we predicted on.
        predict_df = pyfunc_model.predict(df)

        # Checking the PyFunc's predictions are the same as the original model's predictions.
        template = '\nOriginal prediction is "{}", reloaded prediction is "{}"'
        for expec, pred in zip(old_predictions, predict_df['classes']):
            class_id = predict_df['class_ids'][predict_df.loc[predict_df['classes'] == pred].index[0]]
            reloaded_label = SPECIES[class_id]
            print(template.format(expec, reloaded_label))
    finally:
        shutil.rmtree(temp)

2020/03/02 12:39:17 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpx48trhc9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimen

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/variable_scope.py:453: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  shape is not None and isinstance(shape, collections_lib.Sequence) and


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


/opt/conda/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/canned/optimizers.py:102: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  signature = inspect.getargspec(opt.__init__)


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpx48trhc9/model.ckpt.
INFO:tensorflow:loss = 1.40484, step = 0
INFO:tensorflow:global_step/sec: 435.948
INFO:tensorflow:loss = 0.9348641, step = 100 (0.219 sec)
INFO:tensorflow:global_step/sec: 723.32
INFO:tensorflow:loss = 0.88537025, step = 200 (0.139 sec)
INFO:tensorflow:global_step/sec: 746.42
INFO:tensorflow:loss = 0.7834304, step = 300 (0.134 sec)
INFO:tensorflow:global_step/sec: 751.948
INFO:tensorflow:loss = 0.8136801, step = 400 (0.133 sec)
INFO:tensorflow:global_step/sec: 665.671
INFO:tensorflow:loss = 0.68808913, step = 500 (0.159 sec)
INFO:tensorflow:global_step/sec: 628.886
INFO:tensorflow:loss = 0.70802796, step = 600 (0.151 sec)
INFO:tensorflow:global_step/sec: 749.863
INFO:tensorflow:loss = 0.704786, step = 700 (0.134 sec)
INFO:tensorflow:global_step/sec: 741.945
INFO:tensorflow:loss = 0.6784909, ste

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:720: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):


INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27774s
INFO:tensorflow:Finished evaluation at 2020-03-02-12:39:22
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.53333336, average_loss = 0.73073494, global_step = 1000, loss = 0.73073494
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpx48trhc9/model.ckpt-1000

Test set accuracy: 0.533

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpx48trhc9/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (68.5%), expected "Setosa"

Prediction is "Virginica" (43.1%), expected "Versicolor"

Prediction is "Virginica" (57.7%), expected "Virginica"
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely

2020/03/02 12:39:23 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
2020/03/02 12:39:23 INFO mlflow.tensorflow: Validation succeeded!


/tmp/tmp0qiqa6aq/1583152762

Original prediction is "Setosa", reloaded prediction is "Setosa"

Original prediction is "Virginica", reloaded prediction is "Virginica"

Original prediction is "Virginica", reloaded prediction is "Virginica"
